In [1]:
import chromadb

client = chromadb.Client()
collect = client.create_collection(name="my_collection")

In [2]:
collect.add(
    documents=[
        "Doc1 is a description about USA",
        "Doc2 is a description about India"
    ],
    ids=['id1','id2']
)

In [3]:
results = collect.query(query_texts=['Query is about tea'],
                        n_results=2)

results

{'ids': [['id2', 'id1']],
 'embeddings': None,
 'documents': [['Doc2 is a description about India',
   'Doc1 is a description about USA']],
 'uris': None,
 'data': None,
 'metadatas': [[None, None]],
 'distances': [[1.327763557434082, 1.631899118423462]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [4]:
doc = collect.get()
doc['ids']

['id1', 'id2']

In [5]:
collect.delete(ids=doc['ids'])
collect.get()

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'data': None,
 'metadatas': [],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [6]:
collect.add(
    documents=[
        "Doc1 is a description about USA",
        "Doc2 is a description about India"
    ],
    ids=['id1','id2'],
    metadatas=[{'url':'https://en.wikipedia.org/wiki/India'},
                {'url':'https://en.wikipedia.org/wiki/United_States'}]
)

In [7]:
collect.get()

{'ids': ['id1', 'id2'],
 'embeddings': None,
 'documents': ['Doc1 is a description about USA',
  'Doc2 is a description about India'],
 'uris': None,
 'data': None,
 'metadatas': [{'url': 'https://en.wikipedia.org/wiki/India'},
  {'url': 'https://en.wikipedia.org/wiki/United_States'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [8]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [49]:
def clean_text(text):
    # Remove HTML tags
    text = re.sub(r'<[^>]*?>', '', text)
    # Remove URLs
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    # Remove special characters
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text)
    # Replace multiple spaces with a single space
    text = re.sub(r'\s{2,}', ' ', text)
    # Trim leading and trailing whitespace
    text = text.strip()
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

In [50]:
loader = WebBaseLoader("https://ibmglobal.avature.net/en_US/careers/JobDetail?jobId=21839&source=WEB_Search_INDIA")
pg_data = clean_text(loader.load().pop().page_content)
print(pg_data)

Application DeveloperExperience Front End 21839 IBM Skip to content IBM IBM Careers Life IBM Culture Locations Careers blog Teams AI watsonx Cloud Consulting Data Analytics Design UX Enterprise Operations Infrastructure Technology Project and Product Management Research Sales Security Software Engineering Entry Level Internship Career opportunities Career guidance Resources Application process Application FAQs Applicant data privacy Login Back to search results Application DeveloperExperience Front End Mysore Karnataka IndiaConsultingProfessionalEmailXLinkedIn Apply now Introduction In this role youll work in one of our IBM Consulting Client Innovation Centers Delivery Centers where we deliver deep technical and industry expertise to a wide range of public and private sector clients around the world Our delivery centers offer our clients locally based skills and technical expertise to drive innovation and adoption of new technologyIn this role youll work in one of our IBM Consulting Cl

In [51]:
from langchain_core.prompts import PromptTemplate

prompt_ext = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing
    the following keys: 'role', 'experience','skills' and 'description'.
    Only return the valid JSON.
    Skip the think part.
    ### VALID JSON (NO PREAMBLE):
    """
)

In [19]:
pip install ChatOllama

^C
Note: you may need to restart the kernel to use updated packages.


In [52]:
from langchain_ollama.chat_models import ChatOllama 
local_model = "deepseek-r1:8b"
llm = ChatOllama(model=local_model)

In [53]:
chain_ext = prompt_ext | llm
res = chain_ext.invoke(input={'page_data':pg_data})
print(res.content)

<think>
Okay, so I need to figure out how to extract the job posting from the given scraped text. The user provided a chunk of text that seems to be from an IBM career page. My task is to parse this and create a JSON with specific keys: 'role', 'experience', 'skills', and 'description'.

First, I'll start by reading through the text carefully to understand what each section contains. The role is mentioned several times as "Application Developer / Experience Front End" and the job ID 21839. So, the role key seems straightforward.

Next, looking for experience, the text mentions "Entry Level Internship". That should go into the 'experience' field. For skills, I need to find all relevant technical terms. The text lists React JS, HTML/CSS, Redux, TypeScript, Node.js, ES6+, Fastlearner, and critical thinking among others. These will be listed as an array under 'skills'.

The description is a bit longer. It talks about working in IBM Consulting's Client Innovation Centers, delivering technol

In [32]:
import re

In [29]:
def clean_text(text):
    # Remove HTML tags
    text = re.sub(r'<[^>]*?>', '', text)
    # Remove URLs
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    # Remove special characters
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text)
    # Replace multiple spaces with a single space
    text = re.sub(r'\s{2,}', ' ', text)
    # Trim leading and trailing whitespace
    text = text.strip()
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

In [46]:
main = res.content.split('json')
main = main[1]


In [42]:
main = clean_text(main)
main

TypeError: expected string or bytes-like object

In [47]:
from langchain_core.output_parsers import JsonOutputParser

jsonpar = JsonOutputParser()
jsonres = jsonpar.parse(main)
jsonres

{'role': 'Application Developer - Experience Front End',
 'experience': '5-12 years',
 'skills': ['React.js',
  'HTML/CSS',
  'Redux',
  'TypeScript',
  'JavaScript',
  'Node.js',
  'ES5+',
  'Front-end functional testing'],
 'description': "Position within IBM Consulting's Client Innovation Centers, delivering technical expertise and solutions for clients. Responsibilities include feature development, issue resolution, collaboration with stakeholders, and continuous learning in problem-solving and process improvement."}

In [15]:
import pandas as pd

df = pd.read_csv(r'C:\Users\USER\Projects\RAG\ColdEmailGen\app\res\my_portfolio.csv')
df.head()

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [16]:
import uuid

In [17]:
client = chromadb.PersistentClient('vectorstore')
collect = client.get_or_create_collection(name='portfolio')

if not collect.count():
    for _, row in df.iterrows():
        collect.add(documents=row["Techstack"],
                    metadatas={'links':row["Links"]},
                    ids=[str(uuid.uuid4())])

In [18]:
links = collect.query(query_texts=["Experience in Python", "Experience in React"],n_results=2).get('metadatas')
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [23]:
jobs.get('skills')

['React.js',
 'HTML/CSS',
 'Redux',
 'TypeScript',
 'JavaScript',
 'Node.js',
 'ES5+',
 'Front-end functional testing']

In [54]:
links = collect.query(query_texts=jobs.get('skills'),n_results=2).get('metadatas')

In [55]:
links

[[{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/typescript-frontend-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/typescript-frontend-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}]]

In [20]:
jobs = jsonres
jobs['skills']

['React.js',
 'HTML/CSS',
 'Redux',
 'TypeScript',
 'JavaScript',
 'Node.js',
 'ES5+',
 'Front-end functional testing']

In [33]:
email_prompt = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}

    ### INSTRUCTION:
    You are Yash, a business development executive at AtlasLabs. AtlasLabs is an AI research company dedicated to
    facilitating new research about seamless integration of business processes through automated tools.
    Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability,
    process optimization, cost reduction, and heightened overall efficiency.
    Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtlasLabs
    in fulfilling their needs.
    Also add the most relevant ones from the following links to showcase AtlasLabs' portfolio : {link_list}
    Remember, you are Yash, BDE at AtlasLabs.
    Do not provide a preamble.
    ### EMAIL (NO PREAMBLE):
    """
)

In [34]:
chain_email = email_prompt | llm
res = chain_email.invoke({'job_description':str(jobs), "link_list":links})
print(res.content)

<think>
Okay, so I need to help write a cold email for Yash from AtlasLabs to this client about the job opening. Let me start by understanding what the user is asking.

First, I see that the job description is provided as a JSON object with role, experience, skills, and description. The candidate needs to have 5-12 years of experience in application development, specifically in React.js, HTML/CSS, Node.js, ES6+, Redux, TypeScript, front-end testing, Java best practices, and MS Office software.

Yash's role is as a business development executive at AtlasLabs, which is an AI research company focused on seamless business process integration through automated tools. They've helped many enterprises with tailored solutions, leading to scalability, optimization, cost reduction, and efficiency.

The task is to write a cold email to the client about this job, showcasing AtlasLabs' capabilities using relevant portfolio links: one for React.js and another for general Python development. The email

In [35]:
word = res.content

In [36]:
words = word.split('</think>')
print(words[1])



Subject: Opportunity for Application Developer - Experience Front End at AtlasLabs

Hello [Recipient Name],

My name is Yash, from AtlasLabs, an AI research company dedicated to integrating automated tools that enhance business processes. We've helped numerous enterprises optimize their workflows, reduce costs, and improve efficiency through tailored solutions.

I'm reaching out because we're currently looking for a skilled Application Developer with experience in React.js, HTML/CSS, Node.js, etc., as mentioned in the job description. AtlasLabs is expanding its front-end capabilities and would love to explore how your expertise can contribute to our innovative AI solutions.

Take a look at some of our recent projects here: [React Portfolio Link]. We specialize in creating seamless, efficient applications that could complement your technical skills.

Looking forward to hearing from you and discussing potential collaboration opportunities.

Best regards,  
Yash  
Business Development E